In [2]:
import pandas as pd
import requests
import lxml.html as html
import os
import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
#Teniendo la url para hacer el web scraping
urlModalidad='https://admision.unmsm.edu.pe/WebsiteExa_20232/'

def get_modalidad_url(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlModalidad+a['href'][::] for a in soup.select('tbody tr td.text-center a')]
    return urls


In [ ]:
get_modalidad_url(urlModalidad)

In [ ]:
def tabla_modalidad(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        contents = [a.text.lower() for a in soup.select('tbody tr td.text-center a')]
        tabla =pd.DataFrame(contents,columns=['Modalidad']).reset_index()
        tabla.rename(columns={'index': 'id_modalidad'},inplace=True)
    return tabla


In [ ]:
#creando la tabla de modalidades
tabla_modalidad=tabla_modalidad(urlModalidad)

In [ ]:
#Exportando las modalidades de Ingreso 
tabla_modalidad.to_csv('./data/tabla_modalidad.csv',index=False,sep=',')

In [ ]:
#creando la funcion para obtener la los links de cada carrera por modaldidad 
def get_carrera_url(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlAdmision[:-7:]+a['href'][1::] for a in soup.select('tbody tr td.text-center a')]
        return urls

In [ ]:
def tabla_carrera(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        tabla = [[a['href'][4:-7:],a.text.upper()] for a in soup.select('tbody tr td.text-center a')]
        tabla=pd.DataFrame(tabla,columns=['IdCarrera','Carrera'])
        return tabla

In [ ]:
tabla_carrera=tabla_carrera('https://admision.unmsm.edu.pe/WebsiteExa_20232/A.html')
tabla_carrera.IdCarrera=tabla_carrera.IdCarrera.astype(int)

In [ ]:
#opteniendo nuestra tabla de carreras 
tabla_carrera

,IdCarrera,Carrera
0,11,MEDICINA HUMANA
1,12,OBSTETRICIA
2,13,ENFERMERÍA
3,141,TEC. MED. LAB. CLÍNICO Y ANATOMÍA PATOLÓGICA
4,142,TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN
...,...,...
73,192,INGENIERÍA ELÉCTRICA
74,193,INGENIERÍA DE TELECOMUNICACIONES
75,194,INGENIERÍA BIOMÉDICA
76,201,INGENIERÍA DE SISTEMAS


In [ ]:
#exportando nuestra tabla de carreras 
tabla_carrera.to_csv('./data/tabla_carrera.csv',sep=',',index=False)

In [ ]:
url='https://admision.unmsm.edu.pe/WebsiteExa_20232/'

urlsModalidad=get_modalidad_url(url)

In [ ]:
tabla_modalidad

,id_modalidad,Modalidad
0,0,educación básica regular (ebr) y educación bás...
1,1,primeros puestos de educación secundaria
2,2,traslado interno
3,3,graduados o titulados
4,4,traslado externo nacional
5,5,traslado externo internacional
6,6,deportista calificado
7,7,deportistas calificados de alto nivel
8,8,comunidades nativas
9,9,personas con discapacidad


In [ ]:
#en esta funcion se introduce una lista
#devuelve el dataframe de todas las carreraspor modalidad y con id modaldiad
def get_df(urls,num):
    resultados = pd.DataFrame()
    for i, url in enumerate(urls):
        tables = pd.read_html(url, encoding='utf-8')
        table = pd.concat(tables)
        resultados = pd.concat([resultados, table], ignore_index=True)
    resultados['IdModalidad']=num
    return resultados
    

In [ ]:
#recolectamos todas las url opr modalidad de ingreso
urlsModalidad=get_modalidad_url(url)
urlsCarrera=[]
for i , urlModalidad in enumerate(urlsModalidad):
    #guardando las url de carrera
    urlsCarrera.append(get_carrera_url(urlModalidad))


In [ ]:
resultados=get_df(urlsCarrera[7],7)

In [ ]:
resultados

,CODIGO,APELLIDOS Y NOMBRES,ESCUELA PROFESIONAL,PUNTAJE FINAL,MERITO E.P,OBSERVACI&OacuteN,IdModalidad
0,855869,"CORPUNA MONTES, PILAR",MEDICINA HUMANA,409.625,1.0,ALCANZO VACANTE,7
1,183307,"CUBAS CASTILLO, DARIO",NUTRICION,185.125,1.0,ALCANZO VACANTE,7
2,838063,"HIDALGO CASTILLO, MATHIU SEBASTIAN",DERECHO,357.000,1.0,ALCANZO VACANTE,7
3,774916,"RIMARACHE HORNA, EDWIN RONNY",EDUCACIÓN FÍSICA,657.050,1.0,ALCANZO VACANTE,7
4,221415,"FERNÁNDEZ CUBAS, MARCO ADRYAN",INGENIERÍA DE SISTEMAS,374.750,1.0,ALCANZO VACANTE,7
5,289425,"SOSA POMACHAGUA, RHOMINA B´SHOMY",INGENIERÍA DE SISTEMAS,NaN,NaN,AUSENTE,7


In [ ]:
#creando un bucle para extraer la informacion de todos los postulantes ausentes y que ingresaron
todo = pd.DataFrame()
for i in range(len(urlsCarrera)):
    resultado=get_df(urlsCarrera[i],i)
    todo = pd.concat([todo, resultado], ignore_index=True)

In [ ]:
#exportamdo los resultados de examen de admision 
#hacer un for donde extraigamos toda la data y le incluya una columna para especificar su modalidad
todo.to_csv('./data/tabla_resultados.csv',index=False,sep=',')

In [ ]:
todo

,CODIGO,APELLIDOS Y NOMBRES,ESCUELA PROFESIONAL,PUNTAJE FINAL,MERITO E.P,OBSERVACI&OacuteN,IdModalidad
0,855168,"ABAD GRANDA, ANDRE FAHET",MEDICINA HUMANA,774.75,NaN,NaN,0
1,860373,"ABAD NEYRA, ANDREA TAIS",MEDICINA HUMANA,299.0,NaN,NaN,0
2,894207,"ABAD ONCOY, MARIA BELÉN",MEDICINA HUMANA,441.75,NaN,NaN,0
3,875677,"ABAL PRINCIPE, INGRID",MEDICINA HUMANA,530.5,NaN,NaN,0
4,855333,"ABANTO ARAUJO, PERCY ANGEL",MEDICINA HUMANA,584.625,NaN,NaN,0
...,...,...,...,...,...,...,...
25827,255062,"VENANCIO ALCEDO, HENRRY ESAÚ",INGENIERÍA DE SISTEMAS,652.75,NaN,NaN,10
25828,288818,"CARHUA PEREZ, SHERITH OLENKA",INGENIERÍA DE SOFTWARE,542.375,1.0,ALCANZO VACANTE,10
25829,231862,"CCOPA CCACCYA, LUIS MARIANO",INGENIERÍA DE SOFTWARE,436.75,NaN,NaN,10
25830,294871,"GARCIA MARIÑO, CHRISTIAN JESÚS",INGENIERÍA DE SOFTWARE,542.375,2.0,ALCANZO VACANTE,10
